# Model testing

Exploring different LLM inference APIs (e.g. OpenAI, Hugging Face etc.)

In [1]:
import os
import requests

from dotenv import load_dotenv, find_dotenv

from manifesto_qa.models import get_huggingface_llm

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage

/Users/longbe01/Documents/projects/llm-rag/venv-llm-rag/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [2]:
_ = load_dotenv(find_dotenv())

In [3]:
template = """You are an assistant for question-answering tasks. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
    Question: {question}
    Answer:
    """
prompt_template = ChatPromptTemplate.from_template(template)

## HuggingFace

### Test using the requests library

In [31]:
MODEL_ID = "gpt2" # "gpt2" # "deepset/roberta-base-squad2"

In [36]:
headers = {"Authorization": f"Bearer {os.getenv('HUGGING_FACE_API_KEY')}"}

API_URL = f"https://api-inference.huggingface.co/models/{MODEL_ID}"


def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response

payload = {
    "inputs": "What is christmas?"
}

In [37]:
response = query(payload)
response

<Response [200]>

In [38]:
response.json()

[{'generated_text': 'What is christmas? Some known names as Christians forced to work with Christmas taunts & snickering. 6. Christianity extended a horrid riddle to see how it got hurt Jan 16 152 from Cakes Creek - see gridavesfail.html by Cores Izim support book_dismayed_ christmas break c-of-alia_ pictures_76 pictures_femmes_feibel_complete i_ j_ella_beyagle photos_aklar_calinjiev christmas'}]

### Test using Langchain

In [39]:
hf_llm = get_huggingface_llm(
    model_id=MODEL_ID, 
    huggingface_api_token=os.getenv("HUGGING_FACE_API_KEY"),
    max_new_tokens=100
)

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /Users/longbe01/.cache/huggingface/token
Login successful


/Users/longbe01/Documents/projects/llm-rag/venv-llm-rag/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [5]:
# hf_llm.invoke("what is christmas?")

In [16]:
hf_chain = (
    {"question": RunnablePassthrough()}
    | prompt_template
    | hf_llm
    | StrOutputParser()
)

## Cohere

In [3]:
from langchain_cohere import ChatCohere

/Users/longbe01/Documents/projects/llm-rag/venv-llm-rag/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [4]:
# chat = ChatCohere(model="command", cohere_api_key=os.getenv("COHERE_API_KEY"))
chat = ChatCohere(model="command")

In [5]:
chat.invoke("Tell me about who controls Afghanistan")

AIMessage(content='Afghanistan is a country in South Asia. The political leadership of Afghanistan has been contested and unstable throughout recent decades, but the Taliban has been able to seize power since August 2021. \n\nIn October 2021, the Taliban established an interim government led by Mohammed Hassan Akhund, with a promise to serve the nation and ensure the security of citizens. However, the international community has yet to recognize the Taliban-led government.\n\nPreviously, from 2001 to 2021, Afghanistan was led by the Islamic Republic government, which was established in 2004 after the fall of the previous Taliban regime. The President of Afghanistan held the highest rank in the government, which was elected by the people, while the Taliban leadership occupied a largely marginalized role. \n\nThe Taliban is a political and religious group that emerged in the 1990s fighting against the Islamic Republic government, charging that the government was corrupt and neglecting th

In [6]:
messages = [HumanMessage(content="Describe the state of the war in Ukraine in two sentences")]
chat.invoke(messages)

AIMessage(content='The war in Ukraine is a multifaceted and ongoing conflict involving military invasion, paramilitary operations, and cyber warfare. Since the commencement of the war in February 2022, battles have been fiercely contested and shifted in several directions, with neither side gaining a clear advantage. \n\nBoth Ukraine and Russia have undergone successive military strategizing, ensuring that the conflict persists without a clear resolution.', additional_kwargs={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '3dd84674-7817-4e6e-9694-77aa119ac79e', 'token_count': {'input_tokens': 73.0, 'output_tokens': 74.0}}, response_metadata={'documents': None, 'citations': None, 'search_results': None, 'search_queries': None, 'is_search_required': None, 'generation_id': '3dd84674-7817-4e6e-9694-77aa119ac79e', 'token_count': {'input_tokens': 73.0, 'output_tokens': 74.0}}, id='run-446fe29e-b5cc-457f-82a9-

Try it in a chain

In [1]:
from manifesto_qa.models import get_cohere_llm

/Users/longbe01/Documents/projects/llm-rag/venv-llm-rag/lib/python3.10/site-packages/pydantic/_internal/_config.py:341: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
* 'smart_union' has been removed
  warnings.warn(message, UserWarning)


In [9]:
cohere_llm = get_cohere_llm(model_id="command", temperature=0)

In [10]:
cohere_chain = (
    {"question": RunnablePassthrough()}
    | prompt_template
    | cohere_llm
    | StrOutputParser()
)

In [11]:
cohere_chain.invoke({"question": "What is the best spanish cheese?"})

'There are many different types of Spanish cheese, many of which have Protected Designation of Origin status. Some of the most popular Spanish cheeses include: Manchego, a sheep milk cheese from La Mancha with a nutty flavour; Cabrales, a blue cheese from Asturias that is aged in caves; and Idiazabal, a smoked cheese from the Basque Country often served as a topping on pasta.'

## MistralAI

### Using Langchain

In [7]:
from langchain_mistralai.chat_models import ChatMistralAI

In [8]:
# mistral_chat = ChatMistralAI(model_name="mistral-small", api_key=os.getenv("MISTRAL_API_KEY")) 
mistral_chat = ChatMistralAI(model_name="mistral-small")

In [9]:
messages = [HumanMessage(content="Explain who the East India Company were in two sentences")]
mistral_chat.invoke(messages)

AIMessage(content='The East India Company, also known as the British East India Company, was a powerful British joint-stock company that was formed in 1600 with the purpose of trading with the countries of the Indian Ocean rim for spices, textiles, and other valuable commodities. Over time, the company gained significant political and military power in India, eventually controlling large territories and playing a key role in the colonization of the Indian subcontinent by the British Empire.', response_metadata={'token_usage': {'prompt_tokens': 19, 'total_tokens': 115, 'completion_tokens': 96}, 'model': 'mistral-small', 'finish_reason': 'stop'}, id='run-5be5ab66-3bfd-4145-b9f2-6508324cda91-0', usage_metadata={'input_tokens': 19, 'output_tokens': 96, 'total_tokens': 115})

### Using the `mistralai` package

In [1]:
import os
from mistralai import Mistral

In [6]:
api_key = os.environ["MISTRAL_API_KEY"]
model = "mistral-small" # "mistral-large-latest" # "mistral-small"

client = Mistral(api_key=api_key)

In [8]:
chat_response = client.chat.complete(
    model = model,
    messages = [
        {
            "role": "user",
            "content": "What is the best French cheese?",
        },
    ]
)
print(chat_response.choices[0].message.content)

### OpenAI

In [4]:
from manifesto_qa.models import get_openai_llm

In [5]:
openai_llm = get_openai_llm("gpt-3.5-turbo", temperature=0)

In [6]:
openai_llm.invoke("Describe the american war of independence in one sentence")

AIMessage(content='The American War of Independence was a revolutionary conflict fought between the American colonies and Great Britain from 1775 to 1783, resulting in the colonies gaining their independence and forming the United States of America.', response_metadata={'token_usage': {'completion_tokens': 41, 'prompt_tokens': 16, 'total_tokens': 57}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a65312e3-e3a3-4eac-9f6b-1b499da0f1d1-0', usage_metadata={'input_tokens': 16, 'output_tokens': 41, 'total_tokens': 57})

### Using the model loader function

In [4]:
from manifesto_qa.models import get_llm

In [11]:
llm = get_llm("mistral-small", temperature=0)

In [12]:
type(llm)

langchain_mistralai.chat_models.ChatMistralAI

In [13]:
llm

ChatMistralAI(client=<httpx.Client object at 0x147031d80>, async_client=<httpx.AsyncClient object at 0x1470326b0>, mistral_api_key=SecretStr('**********'), temperature=0.0)